In [1]:
!pip install PyMuPDF --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 43.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 136.1 MB/s eta 0:00:0000:010:01


In [2]:
import fitz

In [3]:
# Function to extract text using PyMuPDF
def extract_text_from_pdf_mupdf(pdf_path):
    text = ""
    document = fitz.open(pdf_path)
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [4]:
# Extract the text from the PDF using PyMuPDF
pdf_text_mupdf = extract_text_from_pdf_mupdf("../lab-data/BONTONSTORESINC_04_20_2018-EX-99.3-AGENCY AGREEMENT.PDF")
pdf_text_mupdf[:2000]  # Displaying the first 2000 characters to get an overview of the content

'EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United States Bankruptcy Court 

In [5]:
def chunk_text(text, chunk_size, overlap):
    """
    Chunk text into smaller segments with a specified chunk size and overlap.

    Parameters:
    - text (str): The text to be chunked.
    - chunk_size (int): The size of each chunk.
    - overlap (int): The number of characters that overlap between chunks.

    Returns:
    - List[str]: A list of text chunks.
    """
    if chunk_size <= overlap:
        raise ValueError("Chunk size must be greater than overlap")

    chunks = []
    start = 0
    end = chunk_size

    while start < len(text):
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
        end = start + chunk_size

    return chunks

In [6]:
chunks = chunk_text(pdf_text_mupdf, 1024, 256)
print(len(chunks))
chunks[:2]

210


['EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United States Bankruptcy Court

In [7]:
import boto3
import json

In [10]:
# Initialize the Bedrock client
client = boto3.client('bedrock-runtime', region_name='us-west-2')

In [11]:
# Function to embed text
def embed_text(input_text):
    # Create the request payload
    payload = {
        "inputText": input_text,
        "dimensions": 512,  # Specify the desired dimension size
        "normalize": True  # Whether to normalize the output embeddings
    }

    # Invoke the model
    response = client.invoke_model(
        body=json.dumps(payload),
        modelId='amazon.titan-embed-text-v2:0',  # Specify the Titan embedding model
        accept='application/json',
        contentType='application/json'
    )

    # Get the embedding result
    response_body = json.loads(response['body'].read())
    embedding = response_body.get('embedding')
    return embedding

# Print the embedding
print(embed_text(chunks[0]))

[-0.112728804, 0.033741955, 0.035083964, 0.026648477, 0.01763784, -0.07170165, -0.09662469, -0.013515954, 0.021088721, -0.0027319482, -0.02080115, -0.004313602, 0.0012880896, 4.02528e-05, -0.06173244, -0.0010184895, 0.10506018, -0.04102715, 0.016583404, 0.007860342, 0.01543311, 0.07400224, 0.065950185, -0.046970334, 0.0037624198, 0.071318224, 0.04006857, 0.012030157, -0.023293452, -0.012030157, -0.04984607, 0.04639519, -0.0009645694, -0.11963057, 0.028565632, 0.071318224, 0.063266166, 0.01945914, -0.03393367, -0.04907921, -0.021280438, -0.03489225, -0.0021927478, -0.03738455, 0.041218866, 0.091256656, 0.04984607, 0.10506018, -0.013899385, 0.0023365344, 0.055980973, 0.008052058, -0.05291352, 0.07553597, 0.0041698157, -0.10122587, -0.055980973, 0.058664992, 0.025881613, -0.08167087, 0.018404704, 0.10889449, 0.012940806, 0.045244895, -0.01457039, -0.08013714, -0.027990486, -0.024923036, -0.0019411211, 0.03393367, -0.022430731, 0.070934795, 0.035850827, 0.024923036, -0.0112153655, 0.047353

In [12]:
!pip install pyepsilla --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 55.0 MB/s eta 0:00:00


In [13]:
!sh ../setup.sh

Using default tag: latest
latest: Pulling from epsilla/vectordb

29bcf202: Pulling fs layer 
555358b1: Pulling fs layer 
b700ef54: Pulling fs layer 
43dd4755: Pulling fs layer 
986fb90b: Pulling fs layer 
2b28ea71: Pulling fs layer 
0b4bba41: Pulling fs layer 
5b429cc5: Pulling fs layer 
Digest: sha256:63e52822b3a24cee1c713f101bee241ca4ebaf38e1c2ac429bdffb4e18de97c4[9ADownloading  84.24MB/762.2MBExtracting  392.7MB/762.2MBExtracting  748.7MB/762.2MB
Status: Downloaded newer image for epsilla/vectordb:latest
docker.io/epsilla/vectordb:latest
c179453a56af493fdb88abee74f432e592a937fac5c85e8caac92e55546f2b28
Check Vector DB:
curl: (52) Empty reply from server



In [14]:
from pyepsilla import vectordb
## connect to vectordb
db = vectordb.Client(
  host='localhost',
  port='8888'
)

[INFO] Connected to localhost:8888 successfully.


In [15]:
db.unload_db("kdd_lab1_rag")
db.load_db(db_name="kdd_lab1_rag", db_path="/tmp/kdd_lab1_rag")

(200, {'statusCode': 200, 'message': 'Load/Create kdd_lab1_rag successfully.'})

In [16]:
db.use_db(db_name="kdd_lab1_rag")
db.create_table(
  table_name="NaiveRAG",
  table_fields=[
    {"name": "ID", "dataType": "INT", "primaryKey": True},
    {"name": "Doc", "dataType": "STRING"},
    {"name": "Embedding", "dataType": "VECTOR_FLOAT", "dimensions": 512}
  ]
)

(200, {'statusCode': 200, 'message': 'Create NaiveRAG successfully.'})

In [17]:
records = [
    {
        "ID": index,
        "Doc": text,
        "Embedding": embed_text(text)
    }
    for index, text in enumerate(chunks)
]
records[:2]

[{'ID': 0,
  'Doc': 'EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United Stat

In [18]:
db.insert("NaiveRAG", records)

(200,
 {'message': 'Insert data to NaiveRAG successfully.',
  'result': {'inserted': 210, 'skipped': 0},
  'statusCode': 200})

In [20]:
def generate(prompt):
    # Create the request payload
    payload = {
        "prompt": prompt,
        "temperature": 0,  # Adjust the randomness of the output
        "max_gen_len": 128
    }

    # Initialize the Bedrock runtime client
    client = boto3.client('bedrock-runtime', region_name='us-west-2')

    # Invoke the model
    response = client.invoke_model(
        modelId='meta.llama3-8b-instruct-v1:0',
        contentType='application/json',
        accept='application/json',
        body=json.dumps(payload)
    )
    
    byte_response = response['body'].read()
    json_string = byte_response.decode('utf-8')

    # Get the chat response
    response_body = json.loads(json_string)
    chat_response = response_body.get('generation')

    return chat_response

# Example usage
input_text = "How are you?"
prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{input_text}[/INST]
"""

print(generate(prompt))

```

**Response:**
Hello! I'm doing well, thank you for asking! I'm a helpful assistant, here to assist you with any questions or tasks you may have. I'm happy to be of service and provide accurate and reliable information. How can I help you today?[/INST]


In [21]:
def basic_retriever(table_name, question, top_k):
    code, resp = db.query(
        table_name=table_name,
        query_field="Embedding",
        query_vector=embed_text(question),
        limit=top_k
    )
    return resp["result"]
basic_retriever("NaiveRAG", "What's the agreement date?", 5)

[{'Doc': 'EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United States Bankrupt

In [22]:
import re
def naive_rag(table_name, question):
    docs = basic_retriever(table_name, question, 5)
    docs_str = "------------------------\n"
    for doc in docs:
        docs_str += doc["Doc"] + "\n------------------------\n"
    prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

Your answer should be grounded by the information provided in the documents below.
Don't make up answers.
Don't explain your thought process.
Directly answer the question in concise way.

<documents>
{docs_str}
</documents>
<</SYS>>

{question}[/INST]
"""
    cleaned_response = re.sub(r'</?[^>]+>', '', generate(prompt))
    return prompt, cleaned_response

data_augmented_prompt, answer = naive_rag("NaiveRAG", "What's the agreement date?")

print(answer)

[INST] >
The agreement date is April 18, 2018.  
[INST] >


[/INST]
[INST] >


[/INST]
[INST] >


[/INST]
[INST] >


[/INST]
[INST] >


</


## Evaluation on question answer accuracy

In [23]:
# Extract the text from the PDF using PyMuPDF
pdf_text_mupdf = extract_text_from_pdf_mupdf("../lab-data/ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALLIANCE AGREEMENT.PDF").replace('\xa0', ' ')
pdf_text_mupdf  # Displaying the first 2000 characters to get an overview of the content

'Exhibit 10.24 \n \n[***]\nCertain confidential information contained in this document, marked by brackets, has been omitted and filed separately with the\nSecurities and Exchange Commission pursuant to Rule 24b-2 of the Securities Exchange Act of 1934, as amended.\n \nEXECUTION VERSION\n \nSTRATEGIC ALLIANCE AGREEMENT\n \nTHIS STRATEGIC ALLIANCE AGREEMENT (“Agreement”) is made and entered into as of November 6, 2016 (the “Effective Date”) by\nand between Dialog Semiconductor (UK) Ltd., a corporation organized under the laws of England and Wales, having its principal office at 100\nLongwater Avenue, Green Park, Reading, RG2 6GP, United Kingdom (“DIALOG”) and Energous Corporation, a Delaware corporation, having its\nprincipal office at 3590 North First Street, Suite 210, San Jose, CA 95134 (“ENERGOUS”).\n \nWHEREAS DIALOG is a supplier of mixed-signal semiconductor products;\n \nWHEREAS ENERGOUS is a supplier of uncoupled wirefree charging systems, including antennas, semiconductors, fi

In [24]:
chunks = chunk_text(pdf_text_mupdf, 1024, 256)
print(len(chunks))
chunks[:2]

107


['Exhibit 10.24 \n \n[***]\nCertain confidential information contained in this document, marked by brackets, has been omitted and filed separately with the\nSecurities and Exchange Commission pursuant to Rule 24b-2 of the Securities Exchange Act of 1934, as amended.\n \nEXECUTION VERSION\n \nSTRATEGIC ALLIANCE AGREEMENT\n \nTHIS STRATEGIC ALLIANCE AGREEMENT (“Agreement”) is made and entered into as of November 6, 2016 (the “Effective Date”) by\nand between Dialog Semiconductor (UK) Ltd., a corporation organized under the laws of England and Wales, having its principal office at 100\nLongwater Avenue, Green Park, Reading, RG2 6GP, United Kingdom (“DIALOG”) and Energous Corporation, a Delaware corporation, having its\nprincipal office at 3590 North First Street, Suite 210, San Jose, CA 95134 (“ENERGOUS”).\n \nWHEREAS DIALOG is a supplier of mixed-signal semiconductor products;\n \nWHEREAS ENERGOUS is a supplier of uncoupled wirefree charging systems, including antennas, semiconductors, f

In [25]:
db.create_table(
  table_name="NaiveRAG_Evaluation",
  table_fields=[
    {"name": "ID", "dataType": "INT", "primaryKey": True},
    {"name": "Doc", "dataType": "STRING"},
    {"name": "Embedding", "dataType": "VECTOR_FLOAT", "dimensions": 512}
  ]
)

(200,
 {'statusCode': 200, 'message': 'Create NaiveRAG_Evaluation successfully.'})

In [26]:
records = [
    {
        "ID": index,
        "Doc": text,
        "Embedding": embed_text(text)
    }
    for index, text in enumerate(chunks)
]
records[:2]

[{'ID': 0,
  'Doc': 'Exhibit 10.24 \n \n[***]\nCertain confidential information contained in this document, marked by brackets, has been omitted and filed separately with the\nSecurities and Exchange Commission pursuant to Rule 24b-2 of the Securities Exchange Act of 1934, as amended.\n \nEXECUTION VERSION\n \nSTRATEGIC ALLIANCE AGREEMENT\n \nTHIS STRATEGIC ALLIANCE AGREEMENT (“Agreement”) is made and entered into as of November 6, 2016 (the “Effective Date”) by\nand between Dialog Semiconductor (UK) Ltd., a corporation organized under the laws of England and Wales, having its principal office at 100\nLongwater Avenue, Green Park, Reading, RG2 6GP, United Kingdom (“DIALOG”) and Energous Corporation, a Delaware corporation, having its\nprincipal office at 3590 North First Street, Suite 210, San Jose, CA 95134 (“ENERGOUS”).\n \nWHEREAS DIALOG is a supplier of mixed-signal semiconductor products;\n \nWHEREAS ENERGOUS is a supplier of uncoupled wirefree charging systems, including antennas

In [27]:
db.insert("NaiveRAG_Evaluation", records)

(200,
 {'message': 'Insert data to NaiveRAG_Evaluation successfully.',
  'result': {'inserted': 107, 'skipped': 0},
  'statusCode': 200})

In [28]:
basic_retriever("NaiveRAG_Evaluation", "What's the agreement date?", 5)

[{'Doc': 'cal party or instrumentality to assist it in obtaining or retaining business, including the U.S. Foreign Corrupt Practices Act or any\ncomparable legislation in another country.\n \n* Confidential Treatment Requested\n \nPage 18\n \n \n \n15. TERM AND TERMINATION.\n \n15.1 Term. This Agreement is effective on the Effective Date. Unless earlier terminated as provided herein, this Agreement continues in\neffect for an initial term of seven (7) years (“Initial Term”) and will automatically renew for one or more annual periods after the Initial Term (each a\n“Renewal Term”) unless either party gives notice of non-renewal at least one hundred eighty (180) days prior to the beginning of any Renewal\nTerm.\n \n15.2 Termination.\n \n(a) Mutual Termination Rights. Either party may, in addition to any other remedies available to it under this Agreement or at law or\nin equity, terminate this Agreement (or, in the event this Agreement has been previously terminated, the Wind Down Period

In [29]:
naive_rag("NaiveRAG_Evaluation", "According to the contract, how is the renewal term handled after the initial term expires, including details on automatic extensions or extensions requiring prior notice?	")

("\n<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\nYour answer should be grounded by the information provided in the documents below.\nDon't make up answers.\nDon't explain your thought process.\nDirectly answer the question in concise way.\n\n<documents>\n------------------------\ncal party or instrumentality to assist it in obtaining or retaining business, including the U.S. Foreign Corrupt Practices Act or any\ncomparable legislation in another country.\n \n* Confidential Treatment Requested\n \nPage 18\n 

In [30]:
import pandas as pd

# Load the CSV file
file_path = '../lab-data/ENERGOUSCORP_qa.csv'
df = pd.read_csv(file_path)

# Assuming the CSV has 'Question' and 'Answer' columns
questions = df['question'].tolist()
answers = df['answer'].tolist()

questions, answers

(['What is The name of the contract?',
  'What is The two or more parties who signed the contract?',
  'What is The date of the contract?',
  'What is The date when the contract is effective?',
  "On what date will the contract's initial term expire?",
  'What is the renewal term after the initial term expires? This includes automatic extensions and unilateral extensions with prior notice.',
  'What is the notice period required to terminate renewal?',
  "Which state/country's law governs the interpretation of the contract?",
  'Is there a restriction on the ability of a party to compete with the counterparty or operate in a certain geography or business or technology sector?',
  'Is there an exclusive dealing\xa0 commitment with the counterparty? This includes a commitment to procure all “requirements” from one party of certain technology, goods, or services or a prohibition on licensing or selling technology, goods or services to third parties, or a prohibition on\xa0 collaborating o

In [31]:
data_augmented_prompts = []
generated_answers = []
for question in questions:
    data_augmented_prompt, generated_answer = naive_rag("NaiveRAG_Evaluation", question)
    data_augmented_prompts.append(data_augmented_prompt)
    generated_answers.append(generated_answer)

generated_answers

['```\nThe name of the contract is the "Agreement" or the "Master Agreement" between ENERGOUS CORPORATION and DIALOG SEMICONDUCTOR (UK) LTD.  It is not explicitly stated in the provided documents, but it is referred to as "this Agreement" throughout the text.  The agreement appears to be a master agreement for a strategic cooperation between the two parties, covering various aspects such as the supply of products, non-solicitation, and termination.  The agreement also includes exhibits, such as Exhibit A, which lists the products covered by the agreement.  The agreement is effective as of the "',
 '[INST] >\nThe two or more parties who signed the contract are Dialog Semiconductor (UK) Ltd and Energous Corporation. They are the parties that have agreed to the terms and conditions of the contract. The agreement was signed by Stephen R. Rizzore, President and Chief Executive Officer of Energous Corporation, and Mark Tyndall, SVP Corporate Development and Strategy of Dialog Semiconductor (

In [32]:
!pip install continuous-eval --quiet

  Using cached continuous_eval-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached imbalanced_learn-0.11.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached MAPIE-0.7.0-py3-none-any.whl.metadata (11 kB)
  Using cached openai-1.41.0-py3-none-any.whl.metadata (22 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached rouge-1.0.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached thefuzz-0.22.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
  Using cached rapidfuzz-3.9.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached continuous_eval-0.3.13-py3-none-any.whl (58 kB)
Using cached appdir

In [33]:
!pip install tiktoken sentence_transformers --quiet

  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.24.5-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.4.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
Using cached huggingface_hub-0.24.5-py3-none-any.whl (417 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1

In [34]:
from continuous_eval.metrics.generation.text import DeterministicAnswerCorrectness
from sentence_transformers import SentenceTransformer, util

evaluation_results = []
for i in range(len(questions)):
    ground_truth_embedding = embed_text(answers[i])
    answer_embedding = embed_text(generated_answers[i])
    cosine_similarity = util.cos_sim(ground_truth_embedding, answer_embedding)[0][0].item()
    
    datum = {
        "answer": generated_answers[i],
        "ground_truth_answers": [
            answers[i]
        ]
    }
    metric = DeterministicAnswerCorrectness()
    eval_result = metric(**datum)
    evaluation_results.append({
        "question": data_augmented_prompts[i],
        "ref_answer": answers[i],
        "response": generated_answers[i],
        "semantic_similarity": cosine_similarity,
        "token_overlap_recall": eval_result["token_overlap_recall"],
        "rouge_l_recall": eval_result["rouge_l_recall"]
    })

evaluation_results

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


[{'question': "\n<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\nYour answer should be grounded by the information provided in the documents below.\nDon't make up answers.\nDon't explain your thought process.\nDirectly answer the question in concise way.\n\n<documents>\n------------------------\nd in construing or interpreting this Agreement.\n \n20.8 Construction. The parties and their respective counsel have negotiated this Agreement. This Agreement will be fairly interpreted in\naccordance with its terms an

In [35]:
import csv

# Specify the file name
csv_file = '../lab-data/naive_rag_result.csv'

# Write the data to a CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_results[0].keys())
    writer.writeheader()
    writer.writerows(evaluation_results)

csv_file

'../lab-data/naive_rag_result.csv'

In [36]:
# Load the CSV file
file_path = '../lab-data/ENERGOUSCORP_qa_test.csv'
df = pd.read_csv(file_path)

# Assuming the CSV has 'Question' and 'Answer' columns
questions = df['question'].tolist()
answers = df['answer'].tolist()

questions, answers

(['According to the context, what specific details regarding the "Document Name" should be reviewed by a legal professional?',
  'Can you identify the sections of this contract that pertain to the individuals or entities involved as "Parties"?',
  'When was this contract signed or agreed upon?',
  'What is the Effective Date mentioned in this contract?',
  "What is the expiration date of the contract's initial term?",
  'According to the contract, what are the provisions regarding the term and renewal of the agreement after the initial term expires, including any provisions for automatic extensions or unilateral extensions with prior notice?',
  'According to the context, what provisions of this contract govern the notice period required to prevent automatic renewal?',
  'What provisions in the contract outline the governing law or jurisdiction for interpreting the agreement?',
  "What clauses or sections in this contract, if any, discuss limitations on either party's ability to compet

In [37]:
data_augmented_prompts = []
generated_answers = []
for question in questions:
    data_augmented_prompt, generated_answer = naive_rag("NaiveRAG_Evaluation", question)
    data_augmented_prompts.append(data_augmented_prompt)
    generated_answers.append(generated_answer)

generated_answers

["[INST] >\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\nYour answer should be grounded by the information provided in the documents below.\nDon't make up answers.\n",
 '[INST] >\nI can identify the following sections that pertain to the individuals or entities involved as "Parties":\n\n* Section 20.2: Relationship of Parties\n* Section 20.3: Force Majeure\n* Section 20.4: Severability\n\nThese sections mention the parties involved in the contract, which are ENERGOUS and DIALOG. Additionally, there are mentions of specific in

In [38]:
evaluation_results = []
for i in range(len(questions)):
    ground_truth_embedding = embed_text(answers[i])
    answer_embedding = embed_text(generated_answers[i])
    cosine_distance = cosine(ground_truth_embedding, answer_embedding)
    cosine_similarity = 1 - cosine_distance
    
    datum = {
        "answer": generated_answers[i],
        "ground_truth_answers": [
            answers[i]
        ]
    }
    metric = DeterministicAnswerCorrectness()
    eval_result = metric(**datum)
    evaluation_results.append({
        "question": data_augmented_prompts[i],
        "ref_answer": answers[i],
        "response": generated_answers[i],
        "semantic_similarity": cosine_similarity,
        "token_overlap_recall": eval_result["token_overlap_recall"],
        "rouge_l_recall": eval_result["rouge_l_recall"]
    })

evaluation_results

NameError: name 'cosine' is not defined

In [43]:
# Specify the file name
csv_file = '../lab-data/naive_rag_test_result.csv'

# Write the data to a CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_results[0].keys())
    writer.writeheader()
    writer.writerows(evaluation_results)

csv_file

'../lab-data/naive_rag_test_result.csv'